# Image Classification with Convolution Neural Network (CNN) Assignment

### NOTICE: Please DO NOT MODIFY `print()` method on the assignment 
#### You can add some `print()` methods to check whether it works, but please REMOVE THEM BEFORE SUBMISSION.

1. Construct Kaggle Dataset 
2. Construct a simple CNN
3. Set hyperparameters (optimizer, criterion, num epochs)
4. Write train / validate code
5. Submission guidelines

In [2]:
# Import libraries to use for Deep Learning 

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import datasets, transforms
from torchvision.io import read_image
from torchsummary import summary
import pandas as pd
from PIL import Image
import os 

import cv2 as cv2

In [87]:
!pip install gdown && gdown 'https://drive.google.com/uc?id=1rctM1HDoc24XOcRzsYyTSavaFrvuoKZc' && unzip ./archive.zip -d ./sports

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1rctM1HDoc24XOcRzsYyTSavaFrvuoKZc
To: /content/archive.zip
100% 500M/500M [00:03<00:00, 140MB/s]
Archive:  ./archive.zip
replace ./sports/EfficientNetB3-sports-0.97.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./sports/class_dict.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace ./sports/class_dict.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./sports/images to predict/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

#1. (Assignment) Construct Kaggle Dataset

- Please construct custom dataset dealt in the class.
- Do not use `torch.utils.data.ImageFolder`.
- The structure of Custom Dataset follows 
- Tips) use `sports.csv` files to get data. (it contains filepath, labels and which dataset each data belongs to)
- Tips) use `class_dict.csv` to get the index of each class - numeric values, not string.
- Tips) there are some grayscale (1-channel) images. I recommend to use `cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)` to make it 3-channel image.

```
class CustomDataset(torch.utils.data.Dataset):
    # Inherit torch.utils.data.Dataset class

    def __init__(self,):
        # Initialize the dataset (handling data paths, check input and target data, data augmentation, etc.)

    def __len__(self):
        # Return the number of data or sample in dataset 
    
    def __getitem__(self, index):
        # Return the input and target by index
```


In [3]:
### PLEASE WRITE YOUR CODE BELOW.

class CustomDataset(Dataset):

    def __init__(self, dataset, transform=None):
      # Initialize the dataset (handling data paths, check input and target data, data augmentation, etc.)
      if (dataset != 'train') and (dataset != 'valid') and (dataset != 'test'):
        print('error!')

      label_dict = {}
      label_df = pd.read_csv('./sports/class_dict.csv') # ex: class_index,class,height,width,scale by
                                                              # 0,air hockey,150,150,1
      for index, row in label_df.iterrows():
        label_dict[row['class']] = row['class_index']

      data_dict = {}
      data_df = pd.read_csv('./sports/sports.csv') #                   filepaths           labels data set
                                                     #0         train/air hockey/001.jpg       air hockey    train
      self.data = []
      for index, row in data_df.iterrows():
        if row['data set'] == dataset:
          path = os.path.join('sports', row['filepaths'])
          x = Image.open(path).convert('RGB')
          if transform:
            x = transform(x)
          y = label_dict[row['labels']]
          self.data.append((x, y))

    def __len__(self):
      # Return the number of data or sample in dataset 
      return len(self.data)

    def __getitem__(self, index):
      return self.data[index]
    
### END OF THE CODE.

In [4]:
### PLEASE WRITE YOUR CODE BELOW.
transform=transforms.ToTensor()

train_dataset = CustomDataset('train', transform)
valid_dataset = CustomDataset('valid', transform)
test_dataset  = CustomDataset('test', transform)

### YOU CAN USE ANY TRANSFORMS YOU WANT. MAKE IT RUNNABLE!

### NOTE: Fixed errata - changed train_dataloader, valid_dataloader, test_dataloader to train_loader, valid_loader, test_loader by Seungwoo

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

### END OF THE CODE.

##2. (Assignment) Construct a network - Simple CNN

- Please construct 4 convolution blocks with following sequences.


```
first layer = [2D Conv -> BatchNorm -> ReLU -> Dropout -> Pooling]

- 2D Convolution with 3x3 kernel size, returns output dimension of 16, use stride and padding = 1.
- use whatever pooling you want with 2x2 kernel size and stride of 2.

second layer = [2D Conv -> BatchNorm -> ReLU -> Dropout -> Pooling]

- 2D Convolution with 3x3 kernel size, returns output dimension of 32, use stride and padding = 1.
- use whatever pooling you want with 2x2 kernel size and stride of 2.

thrid layer = [2D Conv -> BatchNorm -> ReLU -> Dropout -> Pooling]

- 2D Convolution with 3x3 kernel size, returns output dimension of 64, use stride and padding = 1.
- use whatever pooling you want with 2x2 kernel size and stride of 2.

fourth layer = [2D Conv -> BatchNorm -> ReLU -> Dropout -> Pooling]

- 2D Convolution with 3x3 kernel size, returns output dimension of 128, use stride and padding = 1.
- use whatever pooling you want with 2x2 kernel size and stride of 2.

classifier = [Linear -> ReLU -> Linear]

- flatten the output tensor.
- first linear layer returns output dimension of 5012
- second linear layer returns output dimension of number of classes

```


In [5]:
### PLEASE WRITE YOUR CODE BELOW.

class SimpleCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3,
                               stride=1, padding=1),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.AvgPool2d(kernel_size=2, stride=2),                   
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3,
                               stride=1, padding=1),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.AvgPool2d(kernel_size=2, stride=2),                   
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,
                               stride=1, padding=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.AvgPool2d(kernel_size=2, stride=2),                   
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,
                               stride=1, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.AvgPool2d(kernel_size=2, stride=2),                   
        )
        self.classifier = nn.Sequential(
          nn.Linear(in_features=25088, out_features=5012),
          nn.ReLU(),
          nn.Linear(in_features=5012, out_features=num_classes)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

### END OF THE CODE.

In [6]:
### NOTE: Fixed errata - changed the below codes by Seungwoo
### model = SimpleCNN(in_channels=3, num_classes=100).to(device)
### summary(model, (3, 256, 256), device='cuda') 

model = SimpleCNN(in_channels=3, num_classes=100).cuda()
summary(model, (3, 224, 224), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             448
       BatchNorm2d-2         [-1, 16, 224, 224]              32
              ReLU-3         [-1, 16, 224, 224]               0
           Dropout-4         [-1, 16, 224, 224]               0
         AvgPool2d-5         [-1, 16, 112, 112]               0
            Conv2d-6         [-1, 32, 112, 112]           4,640
       BatchNorm2d-7         [-1, 32, 112, 112]              64
              ReLU-8         [-1, 32, 112, 112]               0
           Dropout-9         [-1, 32, 112, 112]               0
        AvgPool2d-10           [-1, 32, 56, 56]               0
           Conv2d-11           [-1, 64, 56, 56]          18,496
      BatchNorm2d-12           [-1, 64, 56, 56]             128
             ReLU-13           [-1, 64, 56, 56]               0
          Dropout-14           [-1, 64,

##3. (Assignment) Set hyperparameters

- Set the total number of epochs to be 50 and the learning rate to be 0.001.

- Use any optimizers you want. Please refer [here](https://pytorch.org/docs/stable/optim.html) for furter details.
    - Remember different optimizers have different hyperparameters.
- Set the loss function to be cross entropy loss.

In [7]:
### PLEASE FILL OUT THE HYPERPARAMETERS
### NOTE THAT YOU SHOULD SET DIFFERENT PARAMETERS FOR DIFFERENT OPTIMIZERS.

lr = 0.001
epochs = 50

## OPTIMIZER HYPERPARAMETERS - PLEASE ADD/REMOVE DEPENDS ON OPTIMIZER.
betas = 0
momentum = 0.9

## WHEN USING GPU, PUT `.cuda()` on model and criterion.

model = SimpleCNN(in_channels=3, num_classes=100).cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = torch.nn.CrossEntropyLoss().cuda()

##4. (Assignment) Write train / validation code

- For each epoch, we train and validate the model.
- Note that the validation dataset is not included in test set. 
- Please refer to the following procedure:


    for each epoch:
        model.train()
        get input and target data from train loader
        
        optmizer.zero_grad()             # reset the gradient 
        pred = model(input)

        loss = criterion(pred, target)   # compute the loss
        loss.backward()                  # backprop
        optimizer.step()                 # update the model weights

        model.eval()                     # set the evaluation mode (turn off batchnorm, dropout)
        with torch.no_grad():
            get the input and target data from validation loader

            pred = model(input)
            compute the validation loss  # Optional 
            calculate the validation accuracy
            save the model w.r.t. validation accuracy



In [8]:
def train(model, optimizer, criterion, data_loader, epoch):
    model.train()
    total_loss = 0.0
    for idx, batch in enumerate(data_loader):
        img, target = batch[0].cuda(), batch[1].cuda()

        ### PLEASE WRITE YOUR CODE BELOW.
        # Initialize the optimizer
        optimizer.zero_grad()

        # Make a prediction
        output = model(img)

        # Calculate loss with prediction and target
        loss = criterion(output, target)

        # Compute the gradient
        loss.backward()

        # Update Parameters
        optimizer.step()

        ### END OF THE CODE.

        total_loss += loss.item() 

        if idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch + 1, idx * img.size(0), len(data_loader.dataset),
                100. * idx * img.size(0) / len(data_loader.dataset), 
                loss.data))

    return total_loss / len(data_loader)

def validate(model, criterion, data_loader):
    model.eval()
    val_loss = 0.0
    val_acc = 0.0

    with torch.no_grad():
        for idx, batch in enumerate(data_loader):
            img, target = batch[0].cuda(), batch[1].cuda()

            ### PLEASE WRITE YOUR CODE BELOW.

            # Make a prediction
            output = model(img)

            # Calculate validation loss (although it is optional)
            loss = criterion(output, target)

            # Get the right prediction - make sure naming the prediction as 'predicted' 
            _, predicted = torch.max(output.data, 1)

            ### END OF THE CODE.

            val_loss += loss.item()
            val_acc += (predicted == target).sum().item()

        total_val_acc = val_acc / len(data_loader.dataset)
        print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            val_loss / len(data_loader), val_acc, len(data_loader.dataset),
            100. * total_val_acc))
    
    return total_val_acc

In [9]:
def test(model, data_loader):
    model.eval()
    test_acc = 0.0

    with torch.no_grad():
        for idx, batch in enumerate(data_loader):
            img, target = batch[0].cuda(), batch[1].cuda()

            ### PLEASE WRITE YOUR CODE BELOW.

            # Make a prediction
            output = model(img)

            # Get the right prediction - make sure naming the prediction as 'predicted' 
            _, predicted = torch.max(output.data, 1)

            ### END OF THE CODE.

        print('\n Test set:  Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_acc, len(data_loader.dataset),
            100. * test_acc / len(data_loader.dataset)))

In [ ]:
### NOTE: Fix errata - changed for epoch in range(2) by Seungwoo
train_losses = []
validation_accuracies = []
for epoch in range(epochs):

    ### PLEASE WRITE YOUR CODE BELOW.
    
    # Train your model with train dataloader
    train_loss = train(model, optimizer, criterion, train_loader, epoch)
    train_losses.append(train_loss)

    # Validate your model with validation dataloader
    validation_accuracy = (model, criterion, valid_loader)
    validation_accuracies.append(validation_accuracy)
    ### END OF THE CODE.

Train Epoch: 1 [0/13572 (0%)]	Loss: 3.643844
Train Epoch: 1 [640/13572 (5%)]	Loss: 3.518664
Train Epoch: 1 [1280/13572 (9%)]	Loss: 3.610075
Train Epoch: 1 [1920/13572 (14%)]	Loss: 3.448734
Train Epoch: 1 [2560/13572 (19%)]	Loss: 3.578585
Train Epoch: 1 [3200/13572 (24%)]	Loss: 3.196797
Train Epoch: 1 [3840/13572 (28%)]	Loss: 3.469079
Train Epoch: 1 [4480/13572 (33%)]	Loss: 3.433546
Train Epoch: 1 [5120/13572 (38%)]	Loss: 3.391923
Train Epoch: 1 [5760/13572 (42%)]	Loss: 3.444098
Train Epoch: 1 [6400/13572 (47%)]	Loss: 3.161500
Train Epoch: 1 [7040/13572 (52%)]	Loss: 3.308374
Train Epoch: 1 [7680/13572 (57%)]	Loss: 3.470518
Train Epoch: 1 [8320/13572 (61%)]	Loss: 3.089716
Train Epoch: 1 [8960/13572 (66%)]	Loss: 3.405200
Train Epoch: 1 [9600/13572 (71%)]	Loss: 2.916322
Train Epoch: 1 [10240/13572 (75%)]	Loss: 3.266115
Train Epoch: 1 [10880/13572 (80%)]	Loss: 3.198617
Train Epoch: 1 [11520/13572 (85%)]	Loss: 3.266559
Train Epoch: 1 [12160/13572 (90%)]	Loss: 3.114872
Train Epoch: 1 [12800/1

In [ ]:
# PLEASE WRITE YOUR CODE BELOW.

# Test your model with test dataloader
test(model, criterion, test_loader)

##5. Submission

- Please EXECUTE the below cell and save your file with the output.
- Do not create any other cells below.
- Please save the assignment WITHOUT changing the file name, which should be **CNN_assignment.ipynb**

In [ ]:
### PLEASE EXECUTE THE FOLLOWING CELL BEFORE SUBMITTING YOUR CODE

### DO NOT MODIFY THIS CELL
print(train_dataset[0][0].size())
print(model(torch.rand(1, 3, 224, 224, device='cuda')).size())
test(model, test_loader)
### DO NOT MODIFY THIS CELL